<a href="https://colab.research.google.com/github/aaminrc/Proyect-Intent-recognition/blob/main/ReviewsScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto de Intent Recognition
Addison Reyes, 2021-2026

Todas las librerias usadas en el proyecto

In [ ]:
pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from bs4 import BeautifulSoup
from nltk import FreqDist
import pandas as pd
import numpy as np
import requests
import langid
import string
import pickle
import nltk
import json

In [ ]:
#Dependencias de nltk necesarias para el proyecto
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Web scrapping y recoleccion de toda la data

In [ ]:
def pagina_url(pag_num):
	url = 'https://www.metacritic.com/game/xbox-one/cyberpunk-2077/user-reviews?page='
	pag = requests.Session()
	pag.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

	pagina = pag.get(url + str(pag_num))
	
	soup = BeautifulSoup(pagina.content, 'html.parser')
	return soup


In [ ]:
lang = ['english']
nombres = list()
review = list()
nota = list()
fecha = list()

for i in range(0, 19):
  soup = pagina_url(i)

  nomb = soup.find_all('div', class_="name")
  for i in nomb:
    nombres.append(i.text)
   
  revi = soup.find_all('div', class_="review_body")
  for i in revi:
    review.append(i.text)
    
  no = soup.find_all('div', class_="review_grade")
  for i in no:
    nota.append(i.text)
    
  fech = soup.find_all('div', class_="date")
  for i in fech:
	  fecha.append(i.text)

In [ ]:
r_nombres = list()
r_review = list()
r_nota = list()
r_fecha = list()

for i in range(0, len(nombres)):
  lng = langid.classify(review[i])
  if lng[0] == "en":
    r_nombres.append(nombres[i])
    r_review.append(review[i].lower())
    r_nota.append(nota[i])
    r_fecha.append(fecha[i])

Limpieza de todos los datos y subida a la base de datos

In [ ]:
aux_data = np.array(["", "", "", ""])

pr_datos = list()
pr_nota = list()
vc_data = list()
tokens = list()
datos = list()

for i in range(0, len(r_nombres)):
	aux1 = r_nombres[i]
	aux2 = r_review[i]
	aux3 = r_nota[i]
	aux4 = r_fecha[i]

	aux1 = aux1.strip('\n')
	aux2 = aux2.strip('\n')
	aux3 = aux3.strip('\n')
	aux4 = aux4.strip('\n')

	aux1 = aux1.strip('\t')
	aux2 = aux2.strip('\t')
	aux3 = aux3.strip('\t')
	aux4 = aux4.strip('\t')
	aux2 = aux2.replace("'", "`")

	lng = langid.classify(aux2)
	if lng[0] == "en":
		vc_aux = aux2.lower()
		pr_aux = aux2.lower()
	
		data = np.array([aux1, aux2, aux3, aux4])
		
		tokens_aux = word_tokenize(pr_aux)
		pr_aux = sent_tokenize(pr_aux)

		datos.append(aux2)

		tokens.extend(tokens_aux)
		pr_datos.extend(pr_aux)
		pr_nota.append(1 if int(aux3) >= 3 else 0)
		vc_data.append(vc_aux)

In [ ]:
pr_datos = [token for token in pr_datos if not token.isdigit()]
pr_datos = [token for token in pr_datos if not token in string.punctuation]
tokens = [token for token in tokens if not token.isdigit()]
tokens = [token for token in tokens if not token in string.punctuation]

en_stopwords = stopwords.words('english')
	
pr_datos = [token for token in pr_datos if token not in en_stopwords]
tokens = [token for token in tokens if token not in en_stopwords]

for i in lang:
	stemmer = SnowballStemmer(i)

	pr_datos = [stemmer.stem(token) for token in pr_datos]
	tokens = [stemmer.stem(token) for token in tokens]

for i in pr_datos:
	while pr_datos.count(i) > 1:
		pr_datos.remove(i)
	
for i in tokens:
	while tokens.count(i) > 1:
		tokens.remove(i)

Vectorizacion de los datos y entrenamiento de modelos

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
x = vectorizer.fit_transform(vc_data)
ft_names = vectorizer.get_feature_names()

matrix = x.todense()
list_dense = matrix.tolist()
df = pd.DataFrame(list_dense, columns = ft_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


y_val

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, pr_nota, train_size = 0.75)
#lr = LogisticRegression(C=1)
#lr.fit(x_train, y_train)
#print(accuracy_score(y_val, lr.predict(x_val)))

In [ ]:
#KNearest Neighbors Classifier
knc = KNeighborsClassifier(n_neighbors=3)
knc = knc.fit(x_train, y_train)
print(accuracy_score(y_val, knc.predict(x_val)))

0.761437908496732


In [ ]:
#Random Forest Classifier
rfc = clf = RandomForestClassifier(max_depth=2, random_state=0)
rfc = clf.fit(x_train, y_train)
print(accuracy_score(y_val, rfc.predict(x_val)))

0.5555555555555556


In [ ]:
#Decision Tree Classifier
dtc = DecisionTreeClassifier()
dtc = dtc.fit(x_train, y_train)
print(accuracy_score(y_val, dtc.predict(x_val)))

0.7418300653594772


In [ ]:
#Naive Bayes Classifier
nbc = GaussianNB()
nbc = nbc.fit(x_train.toarray(), y_train)
print(accuracy_score(y_val, nbc.predict(x_val.toarray())))

0.6274509803921569


In [ ]:
#MultinomialNB
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)
print(accuracy_score(y_val, mnb.predict(x_val)))

0.6372549019607843


In [ ]:
#AdaBoostClassifier   <--
ac = AdaBoostClassifier(n_estimators = 1800, learning_rate = 1)
ac = ac.fit(x_train, y_train)
print(accuracy_score(y_val, ac.predict(x_val)))

0.8235294117647058


In [ ]:
pickle.dump(ac, open('ac.model', mode = 'wb'))
pickle.dump(vectorizer, open('vectorizer.model', mode = 'wb'))

In [ ]:
y_true = y_val
y_pred = ac.predict(x_val)

In [ ]:
cr = classification_report(y_true, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.84      0.73      0.78       132
           1       0.82      0.89      0.85       174

    accuracy                           0.82       306
   macro avg       0.83      0.81      0.82       306
weighted avg       0.82      0.82      0.82       306



In [ ]:
print(confusion_matrix(y_true, y_pred))

[[ 97  35]
 [ 19 155]]


Codigo de la api externa

In [ ]:
import sklearn
import pickle

if __name__ == '__main__':
	ac_mod = pickle.load(open('ac.model', mode = 'rb'))   #red binaria
	vc_mod = pickle.load(open('vectorizer.model', mode = 'rb'))

	review = input("Type your review: ")

	pred = ac_mod.predict(vc_mod.transform([review]))

	if pred[0] == 1:
		print("Your review is positive ",pred)
	else:
		print("Your review is negative ",pred)

Type your review: I love this game is very boring
Your review is negative  [0]
